# Импорт

In [20]:
!pip install catboost
!pip install pymorphy2

In [21]:
# при работе через Google Colab
# прописать путь до всех файлов py
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Учёба/8 семестр/диплом/Проект/VKR/Блокноты')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import AnalysisFunc
import BaseFuncs
import PredictionSplit
import PredictionResults

In [23]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, confusion_matrix, f1_score, classification_report, recall_score, mean_absolute_percentage_error
# from sklearn import preprocessing

import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np


# Загрузка данных

In [24]:
from google.colab import drive
drive.mount('/content/drive')
data = BaseFuncs.GetDataFrameListFromFileConverted("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Преобразованные/ПиОА тест div20kr.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# загрузка данных через виндовс
data = BaseFuncs.GetDataFrameListFromFileConverted(BaseFuncs.OpenFileNameWindows())


In [31]:
data.drop_duplicates(keep='last')
data = AnalysisFunc.MakeFloat(data, ["Пол"])

ex = "Контрольная работа 3"

# Словари параметров

In [26]:
parameters_AdaBoostRegressor = {
     'n_estimators': range(1, 15),
    'learning_rate' : [0.01, 0.05, 0.1, 0.5],
    'loss' : ['linear', 'square', 'exponential'],
}

parameters_RandomForestRegressor = {'bootstrap': [True], 'max_depth': range(1, 15),
                'max_features': ['sqrt', 'log2', None],
                'min_samples_leaf': [1,2,3],
                'min_samples_split': [1,2,3],
                'n_estimators': range(1, 15)}

parameters_DecisionTreeRegressor={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': range(1, 15),
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1,2,3],
                         'min_samples_split': [1,2,3],
                         'splitter': ['best', 'random']}

parameters_KNeighborsRegressor = {
    'n_neighbors': range(1,16),
    'leaf_size': range(20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}

parameters_CatBoostRegressor = {
    'iterations': range(1,251, 25),
    'silent': [True],
    'depth': range(1, 10, 1),
    'allow_writing_files': [False],
    'learning_rate': [0.01, 0.05, 0.1]
}



parameters_AdaBoostClassifier = {
    'n_estimators':range(1,50),
    'base_estimator':[None, DecisionTreeClassifier(), SVC(), LogisticRegression()],
    'learning_rate': [0.5, 1.0, 2.0, ]
}

parameters_RandomForestClassifier={'max_depth':(1,15, 1),
              'n_estimators':[10,100,200],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }
parameters_DecisionTreeClassifier={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 15),
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1,2,3],
                         'min_samples_split': [1,2,3],
                         'splitter': ['best', 'random']}

parameters_KNeighborsClassifier = {
    'n_neighbors': range(1,16),
    'leaf_size': range(20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}


parameters_CatBoostClassifier = {
    'iterations': range(1,251, 10),
    'silent': [True],
    'allow_writing_files': [False],
    'learning_rate': [0.01, 0.05, 0.1, 0.5]
}

# Определение параметров контрольной



In [ ]:
#  найти параметры регрессии
X, Y = PredictionSplit.SplitTestTrainTest(data, ex, ["Пол", "Оценка"]+AnalysisFunc.before)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)


grid_search=GridSearchCV(cv=5, estimator=AdaBoostRegressor(), n_jobs=-1,
             param_grid=parameters_AdaBoostRegressor,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"AdaBoostRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )


grid_search=GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid=parameters_RandomForestRegressor,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"RandomForestRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )



grid_search=GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid=parameters_DecisionTreeRegressor,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"DecisionTreeRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20)



grid_search=GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid=parameters_KNeighborsRegressor,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"KNeighborsRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20)



grid_search = GridSearchCV(
    estimator = CatBoostRegressor(),
    param_grid = parameters_CatBoostRegressor,
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred_KNN1 =pred.predict(X_test)
print(f"CatBoostRegressor best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )

Fitting 5 folds for each of 168 candidates, totalling 840 fits
AdaBoostRegressor parameters: {'learning_rate': 0.01, 'loss': 'square', 'n_estimators': 12}
Best Score: -3.453942699586733
Fitting 5 folds for each of 5292 candidates, totalling 26460 fits
RandomForestRegressor parameters: {'bootstrap': True, 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 6}
Best Score: -3.400402267367352
Fitting 5 folds for each of 3024 candidates, totalling 15120 fits
DecisionTreeRegressor parameters: {'criterion': 'absolute_error', 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'splitter': 'random'}
Best Score: -3.4058494195688227
Fitting 5 folds for each of 2400 candidates, totalling 12000 fits
KNeighborsRegressor parameters: {'leaf_size': 34, 'metric': 'minkowski', 'n_neighbors': 14, 'p': 1, 'weights': 'uniform'}
Best Score: -3.461290744689252


# Определение параметров сдачи

In [28]:
#найти наибольшую глубину
def FindParametersClasses(X, Y, error_type):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

    grid_search = GridSearchCV( estimator=AdaBoostClassifier(), param_grid=parameters_AdaBoostClassifier, scoring = error_type,
        n_jobs = -1,
        cv = 5)
    pred = grid_search.fit(X_train, y_train)
    y_pred = pred.predict(X_test)
    print(f"AdaBoostClassifier parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )



    grid_search = GridSearchCV(
        estimator = RandomForestClassifier(),
        param_grid = parameters_RandomForestClassifier,
        scoring =error_type,
        n_jobs = -1,
        cv = 5
    )
    pred=grid_search.fit(X_train, y_train)
    y_pred = pred.predict(X_test)
    print(f"RandomForestClassifier parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )

    grid_search = GridSearchCV(
        estimator = DecisionTreeClassifier(),
        param_grid = parameters_DecisionTreeClassifier,
        scoring =error_type,
        n_jobs = -1,
        cv = 5
    )
    pred=grid_search.fit(X_train, y_train)
    y_pred = pred.predict(X_test)
    print(f"DecisionTreeClassifier parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )


    grid_search = GridSearchCV(
        estimator = KNeighborsClassifier(),
        param_grid = parameters_KNeighborsClassifier,
        scoring = error_type,
        n_jobs = -1,
        cv = 5
    )
    pred=grid_search.fit(X_train, y_train)
    y_pred_KNN1 =pred.predict(X_test)
    print(f"Knn best parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )



    grid_search = GridSearchCV(
        estimator = CatBoostClassifier(),
        param_grid = parameters_CatBoostClassifier,
        scoring = error_type,
        n_jobs = -1,
        cv = 5
    )
    pred=grid_search.fit(X_train, y_train)
    y_pred_KNN1 =pred.predict(X_test)
    print(f"CatBoostClassifier best parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )


In [30]:
X, Y = PredictionSplit.SplitTestTrainPass(data, ex, ["Пол", "Оценка"]+AnalysisFunc.before)

FindParametersClasses(X, Y, 'recall')

AdaBoostClassifier parameters: {'base_estimator': LogisticRegression(), 'learning_rate': 0.5, 'n_estimators': 1}
Best Score: 1.0
RandomForestClassifier parameters: {'max_depth': 1, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Score: 0.970216049382716
DecisionTreeClassifier parameters: {'criterion': 'log_loss', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'splitter': 'random'}
Best Score: 0.9478703703703705
Knn best parameters: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 13, 'p': 1, 'weights': 'distance'}
Best Score: 0.8985185185185184
CatBoostClassifier best parameters: {'allow_writing_files': False, 'iterations': 111, 'learning_rate': 0.1, 'silent': True}
Best Score: 0.9282716049382715


# Определение параметров контрольной как классов

In [32]:
#найти наибольшую глубину
X, Y = PredictionSplit.SplitTestTrainTest(data, ex, ["Пол", "Оценка"])
Y = Y*20
Y = Y.round(0)
FindParametersClasses(X, Y, 'neg_mean_absolute_error')

AdaBoostClassifier parameters: {'base_estimator': LogisticRegression(), 'learning_rate': 1.0, 'n_estimators': 45}
Best Score: -3.8732891100055284
RandomForestClassifier parameters: {'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 100}
Best Score: -3.76057490326147
DecisionTreeClassifier parameters: {'criterion': 'log_loss', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 3, 'splitter': 'random'}
Best Score: -3.883095632946379
Knn best parameters: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 8, 'p': 1, 'weights': 'distance'}
Best Score: -3.634118297401879
CatBoostClassifier best parameters: {'allow_writing_files': False, 'iterations': 181, 'learning_rate': 0.05, 'silent': True}
Best Score: -3.5805638474295187
